# S3W8D3: 结构化输出 (Structured Output)

如果说前两天的内容是在教模型“怎么思考”（Prompting），那么今天的内容就是教模型“**怎么守规矩**”。

在实际工程中，老板和前端开发不需要一个会吟诗作对的 AI，他们只需要一个能吐出标准 JSON 的 API，这样才能存入数据库或显示在网页上。

**🎯 今日目标**

1. **痛点体验**：看 LLM 如何“不听话”地输出多余的废话。
2. **暴力破解**：使用 Regex (正则表达式) 强行提取 JSON。
3. **优雅方案**：结合 **Pydantic** 定义数据 Schema，并利用 Prompt 约束输出。
4. **工程封装**：编写 `src/llm/parsers.py`。

## 📖 1 理论知识讲解 (Theory & Analogy)

> **🧬 医学图像类比：影像报告 vs. DICOM Header**
> * **非结构化 (Natural Language)**：就像医生的口述录音：“病人肺部有一个大概 3 厘米的阴影，位置偏左上...”
> * 这就好比 LLM 说：“根据案情，我觉得张三是盗窃，金额大概是 5000 吧。”
> * **后果**：代码无法处理（`if "大概 3 厘米" > 2.0: ...` 肯定报错）。
> 
> 
> * **结构化 (JSON/Schema)**：就像 **DICOM 文件头**或结构化电子病历。
> * `{"lesion_size_mm": 30, "location": "LUL", "probability": 0.85}`
> * **后果**：代码可以直接读取 `data['lesion_size_mm']` 进行自动分诊。
> 
> 
> 
> 

我们的目标，就是强迫那个爱说话的“医生”（LLM），把结果填进我们的“表格”（JSON）里，而不是口述。

## 💻 3. 代码实现 (Notebook 实验)

请在 Notebook 中依次运行。

### Step 1: 问题的产生 (The "Chatty" Model)

我们试图让模型提取案件信息。

In [3]:
import json
import sys
import os
sys.path.append(os.path.abspath("../../")) 
from src.llm.client import LLMClient

client = LLMClient()

case = "嫌疑人张三趁夜色溜进李四家，偷走现金 5000 元和一部价值 2000 元的手机。"

prompt = f"""
请分析以下案件，提取：嫌疑人姓名、被盗总金额、犯罪类型。
请以 JSON 格式输出。

案件：{case}
"""

response = client.generate(prompt)
print(f"--- 原始回复 ---\n{response}")

# 尝试解析
try:
    data = json.loads(response)
    print("解析成功！")
except json.JSONDecodeError:
    print("\n❌ 解析失败！模型可能输出了多余的文字 (Markdown 标记等)")

--- 原始回复 ---
{
  "嫌疑人姓名": "张三",
  "被盗总金额": 7000,
  "犯罪类型": "盗窃"
}
解析成功！


> **🔴 常见翻车现场**：
> 模型经常会输出：
> ```markdown
> ```json
> {
>     "name": "张三",
>     ...
> }
> 
> ```
> 
> 
> 这里的 JSON 数据供参考。
> ```
> 这种带有 Markdown 标记 (` ```json `) 或者前后废话的字符串，直接用 `json.loads()` 会报错。
> 
> ```
> 
> 

### Step 2: 正则表达式 - 暴力清洗 (The Regex Fix)

这是最通用的“创可贴”修复法。不管模型说了什么废话，我们只抓取 `{` 和 `}` 之间的内容。


In [4]:
import re

def extract_json_with_regex(text):
    # 匹配最外层的 {}
    # DOTALL 模式让 . 可以匹配换行符
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        json_str = match.group()
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            return None
    return None

data = extract_json_with_regex(response)
print(f"--- Regex 提取结果 ---\n{data}")
# 如果 Step 1 输出带 Markdown，这里通常能救回来

--- Regex 提取结果 ---
{'嫌疑人姓名': '张三', '被盗总金额': 7000, '犯罪类型': '盗窃'}


### Step 3: Pydantic - 定义“完美模具” (The Schema Way)

为了让模型更听话，我们直接把 JSON 的定义（Schema）喂给它。

In [5]:
from pydantic import BaseModel, Field

# 1. 定义数据结构 (类比：定义 C++ 的 Struct 或 SQL 表结构)
class CrimeCase(BaseModel):
    suspect: str = Field(description="嫌疑人的姓名")
    amount: float = Field(description="涉及的总金额，纯数字")
    crime_type: str = Field(description="犯罪类型，如：盗窃、抢劫")
    summary: str = Field(description="一句话案情摘要，不超过20字")

# 2. 获取 Schema (这就是我们要喂给 Prompt 的'表格')
schema_str = json.dumps(CrimeCase.model_json_schema(), indent=2, ensure_ascii=False)
print(f"--- Pydantic Schema ---\n{schema_str}")

# 3. 构造精准 Prompt
prompt_with_schema = f"""
请分析案件并提取信息。
**必须严格遵守以下 JSON 格式输出，不要输出任何 Markdown 标记或额外解释：**

{schema_str}

案件：{case}
"""

response_structured = client.generate(prompt_with_schema, temperature=0.0) # 结构化任务记得把 T 降为 0
print(f"\n--- 结构化回复 ---\n{response_structured}")

# 4. 验证并转回对象
final_data = extract_json_with_regex(response_structured)
if final_data:
    try:
        # Pydantic 还能帮我们检查数据类型 (比如 amount 是不是 float)
        case_obj = CrimeCase(**final_data)
        print("\n✅ Pydantic 验证通过！")
        print(f"嫌疑人: {case_obj.suspect}")
        print(f"金额: {case_obj.amount}")
    except Exception as e:
        print(f"❌ 数据类型错误: {e}")

--- Pydantic Schema ---
{
  "properties": {
    "suspect": {
      "description": "嫌疑人的姓名",
      "title": "Suspect",
      "type": "string"
    },
    "amount": {
      "description": "涉及的总金额，纯数字",
      "title": "Amount",
      "type": "number"
    },
    "crime_type": {
      "description": "犯罪类型，如：盗窃、抢劫",
      "title": "Crime Type",
      "type": "string"
    },
    "summary": {
      "description": "一句话案情摘要，不超过20字",
      "title": "Summary",
      "type": "string"
    }
  },
  "required": [
    "suspect",
    "amount",
    "crime_type",
    "summary"
  ],
  "title": "CrimeCase",
  "type": "object"
}

--- 结构化回复 ---
{
  "suspect": "张三",
  "amount": 7000,
  "crime_type": "盗窃",
  "summary": "张三盗窃李四家现金和手机"
}

✅ Pydantic 验证通过！
嫌疑人: 张三
金额: 7000.0


## 🧠 4. 深度理论 & 面试必问 (Deep Dive)

### 🔥 Q1: 如果模型生成的 JSON 少了一个括号，工程上怎么容错？

这个问题非常经典。你有三层防御体系：

1. **Level 1: 提示工程 (Prompting)**
* 在 System Prompt 中强调：*"You are a JSON generator. Do not output anything else."*
* 使用 **One-shot**：给一个合法的 JSON 示例。
* **JSON Mode**: 现在很多 API (OpenAI/DeepSeek) 支持参数 `response_format={"type": "json_object"}`，这在底层强行限制了 Logits 的生成，极大降低语法错误率。


2. **Level 2: 规则修复 (Rule-based Repair)**
* **Regex**: 就像刚才写的，只提取 `{}`。
* **第三方库**: 使用 `json_repair` 库（Python 包），它可以自动补全缺失的 `}` 或引号。


3. **Level 3: 自愈循环 (Self-Correction / Retry) —— 杀手锏**
* 如果 `json.loads` 报错，**捕获错误信息**，把“错误信息”和“错误的 JSON”一起扔回给 LLM。
* **Prompt**: *"你刚才生成的 JSON 解析报错了：Expectng value at line 1 column 10。请修复它并重新输出。"*
* 这利用了 LLM 强大的纠错能力。LangChain 的 `OutputParser` 就内置了这种 Retry 机制。

## 🏗️ 5. 工程封装 (Engineering)

让我们把提取逻辑封装到 `src/llm/parsers.py`。

```python
# src/llm/parsers.py
import re
import json
from typing import Optional, Any, Type
from pydantic import BaseModel

class BaseOutputParser:
    def parse(self, text: str) -> Any:
        raise NotImplementedError

class JsonOutputParser(BaseOutputParser):
    def __init__(self, pydantic_model: Optional[Type[BaseModel]] = None):
        """
        :param pydantic_model: 如果提供，会用它来验证数据结构
        """
        self.pydantic_model = pydantic_model

    def parse(self, text: str) -> dict:
        # 1. 尝试清洗 Markdown 标记 (```json ... ```)
        cleaned_text = text.strip()
        # 匹配 ```json ... ``` 或 ``` ... ```
        match = re.search(r"```(json)?(.*?)```", cleaned_text, re.DOTALL)
        if match:
            cleaned_text = match.group(2).strip()
        
        # 2. 如果还有多余字符，尝试 Regex 提取最外层 {}
        match_bracket = re.search(r"\{.*\}", cleaned_text, re.DOTALL)
        if match_bracket:
            cleaned_text = match_bracket.group()

        # 3. 解析 JSON
        try:
            data = json.loads(cleaned_text)
        except json.JSONDecodeError as e:
            raise ValueError(f"JSON 解析失败: {e}\n原文: {text}")

        # 4. Pydantic 验证
        if self.pydantic_model:
            try:
                obj = self.pydantic_model(**data)
                return obj.model_dump() # 转回 dict，或者直接返回 obj
            except Exception as e:
                raise ValueError(f"Schema 验证失败: {e}")
        
        return data

```

## 🧩 6. 今日 LeetCode 练习

处理结构化输出经常涉及字符串解析。

* **题目**: **[String to Integer (atoi) (LC 8)](../../LeetCode%20practice/1-50.ipynb)** - 中等
* **关联**: 这道题就是让你写一个简单的 Parser。
* 需要丢弃前导空格。
* 处理正负号。
* 遇到非数字字符停止。
* 这和我们从 LLM 的废话中提取 JSON 的逻辑（状态机思维）非常像。

## ✅ 7. 今日任务总结

* [ ] 在 Notebook 中体验了 LLM 输出 JSON 时的“不听话”。
* [ ] 实现了 **Regex** 提取法。
* [ ] 使用 **Pydantic** 生成了 JSON Schema，并以此约束 Prompt。
* [ ] **面试准备**：掌握了 JSON 容错的三层体系（Prompt, Rule, Retry）。
* [ ] **代码封装**：完成了 `src/llm/parsers.py`。